In [2]:
import shap
import tensorflow as tf
import tensorflow.keras.backend
from keras.models import model_from_json
import sys
import time
from dataset_prepare import prepare_data
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.utils import np_utils as npu
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import shap
from IPython.display import display
import pandas as pd 
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from feature_selector import FeatureSelector

def clean_nominals_and_create_our_datasets(train_set_df,test_set_df):
    #Get nominal columns
    nominal_cols = train_set_df.select_dtypes(include='object').columns.tolist()
    #Turn nominal to numeric train and testX
    for nom in nominal_cols:
        le = LabelEncoder()
        le.fit(train_set_df[nom])
        train_set_df[nom]=le.transform(train_set_df[nom])
        testEnc = LabelEncoder()
        testEnc.classes_=le.classes_
        testEnc.fit(test_set_df[nom])
        test_set_df[nom] =testEnc.transform(test_set_df[nom])
        #print(dict(zip(testEnc.classes_, testEnc.transform(testEnc.classes_))))
    classes = dict(zip(testEnc.classes_, testEnc.transform(testEnc.classes_)))

    # drop the nominal columns from the initial set
    train_set_df_y = train_set_df.attack_cat
    train_Y = np.array(train_set_df_y)
    train_Y = train_Y.reshape((train_Y.shape[0],1))
    train_set_df = train_set_df.drop(["attack_cat","label"], axis=1)
    train_X = np.array(train_set_df)

    test_set_df_y = test_set_df.attack_cat
    test_set_df_y1 = test_set_df.label
    test_Y = np.array(test_set_df_y)
    test_Y = test_Y.reshape((test_Y.shape[0],1))
    test_set_df = test_set_df.drop(["attack_cat","label"], axis=1)
    test_X = np.array(test_set_df)

    labels = np.array(test_set_df_y1)
    labels = labels.reshape((labels.shape[0],1))
    
    return train_X,train_Y,test_X,test_Y,test_set_df,labels, classes

def prepare_data(): 
    """
    This function is the main of this module. calls the above functions in order to read/clean/save
    our data in usable form.
    I created this function to use dataset_prepare.py as a Python module in our main program.
    
    Return values: training X,Y dataset and testing X,Y dataset
    """

    # read our csv files
    features_df = pd.read_csv("UNSW_NB15_features.csv",encoding = "ISO-8859-1")
    training_df = pd.read_csv("training.csv").drop("id",axis=1)
    testing_df = pd.read_csv("testing.csv").drop("id",axis=1)

    fs = FeatureSelector(data = training_df)
    fs.identify_collinear(correlation_threshold=0.85)
    training_df = fs.remove(methods = ['collinear'],keep_one_hot = True)
    columnlist = list(training_df)
    testing_df = testing_df[columnlist]
    
    training_df = training_df.sample(frac=1)
    testing_df = testing_df.sample(frac=1)
    train_x,train_y,test_x,test_y, testing_df, labels, classes = clean_nominals_and_create_our_datasets(training_df,testing_df)

    training_df = training_df.drop(["attack_cat","label"], axis=1)
    training_df = np.array(list(training_df))
    #print("The features we will use are: ")
    #for i in range(len(training_df)):
    #    print("["+ str(i) +"]: " + str(training_df[i]))

    return train_x,train_y,test_x,test_y,testing_df,labels, classes, training_df

def load_model(name):
    #load json and create model
    json_file = open(name+".json","r")
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(name+".h5")
    model.compile(loss='mean_squared_error', optimizer='adam')
    print("Loaded model...")
    return model

model = load_model('idsmodel')
train_x,train_y,test_x,test_y ,testxdf, labels, classes, traindf= prepare_data()
train_y = train_y.reshape((train_y.shape[0],))
print(tensorflow.__version__)
explainer = shap.DeepExplainer(model, train_x)

shap_values = explainer.shap_values(test_x[:10])

shap.initjs()

classnames = [x for x in classes.keys()]
display(shap.summary_plot(shap_values, testxdf, class_names=classnames))

Loaded model...
No labels provided. Feature importance based methods are not available.
14 features with a correlation magnitude greater than 0.85.

Removed 14 features.
1.13.1


TypeError: shap_values() missing 1 required positional argument: 'X'